In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import vis

from collections import Counter
from pprint import pprint

import numpy as np
from nltk.tree import Tree
from nltk import word_tokenize

not use stanford CoreNLP client!


[nltk_data] Error loading sentiwordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>


In [ ]:
is_news = 0

l = 5
t = 18
m = "c_v"
i = 200
min_df = 3

# load
if is_news:
    _raw = persister.load_json(configs.RAWNEWS)
    parse = persister.read_parse()
    _input = persister.read_input(configs.NEWSINPUT)
    terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
    model = persister.load_model(configs.NEWSMODEL.format(l, t, m, i))
    vec = persister.load_model(configs.NEWSVEC.format(min_df))
    print("load news")
else:
    _raw = persister.load_json(configs.RAWABSTRACT)
    _input = persister.read_input(configs.ABSTRACTINPUT)
    terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA)
    parse = persister.read_parse(configs.ABSTRACTPARSE)
    model = persister.load_model(configs.ABSTRACTMODEL.format(l, t, m, i))
    vec = persister.load_model(configs.ABSVEC.format(min_df))
    print("load abs")
tf = vec.fit_transform(_input)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)
df_top_words, df_top_docs = lda.pd_topics_vis(top_terms, top_docs)

In [ ]:
len(terms)

In [ ]:
print("docs distribution:",dict(Counter(lda.get_dominant_topic(doc_topic))))

In [ ]:
print("top terms info:")
df_top_words

In [ ]:
print("top docs info:")
df_top_docs

In [ ]:
# vis.pyLDA(topic_word, doc_topic, [len(s) for s in [word_tokenize(corp) for corp in _input]], vec.get_feature_names(), np.array(sum(tf).todense())[0])

In [ ]:
for term in top_terms[0]:
    for news in top_docs[0]:
        _idx = news[0]
        if type(parse[_idx]) is str:
            print(_idx, "parse", "err")
            continue
        for sent_idx,sent in enumerate(parse[_idx]["sentences"]):
            topic_word_idxs = []
            for idx, i in enumerate(sent["tokens"]):
                if i["lemma"].lower() == term[0]:
                    topic_word_idxs.append(idx)
            for i in topic_word_idxs:
                print(sent_idx, i, _idx)
                rs = relation.extract_word_relation_from_sent(i, sent["enhancedDependencies"])
                for r in rs:
                    print(relation.convert_relation2str(r))